<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=d2ecadfe7dab74f97cfd8f641243ed8b14d2fec187c3d29f431af7502457bf43
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pandas import json_normalize
import pyspark.sql.functions as F
import pyspark.sql.types as T
import random
import requests
import json
import os
import pandas as pd
import datetime

In [ ]:
# spark=SparkSession.builder.appName('test').config("spark.sql.shuffle.partitions", "4").getOrCreate()

In [3]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [ ]:
# url = "https://api.carbonintensity.org.uk/intensity"

# while switch:
#   response = requests.get(url)
#   json_normalize(response.json())
#   message = response.json()
#   json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
#                                                   "intensity.actual": "intensity_actual",\
#                                                   "intensity.percentile": "intensity_percentile",\
#                                                   "intensity.index": "intensity_index"\
#                                                   }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)


In [ ]:
# read lines from file
def flat_json(line):
  # convert line into json
  line_json = json.loads(line)
# json normalize
  flatline = pd.json_normalize(line_json,sep="_")
  return flatline


#Inputfile generation

In [4]:
%rm -rf input
%mkdir input

In [5]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-16 06:57:36'

In [25]:
i = 0
s = requests.Session()
url = "https://stream.wikimedia.org/v2/stream/recentchange"
file_number = str(random.randint(1,10000))
with s.get(url, stream=True) as resp:
    for line in resp.iter_lines(1024,decode_unicode=True):
        # Check if line has some bytes in it
        if line:
            # convert line into json
            # get data lines only
            if line[:4] == "data":
                # print(line[5:])
                json_line = json.loads(line[5:])
                flatlines = pd.json_normalize(json_line,sep="_")
                with open("input/rc"+str(random.randint(1,10000))+".json","a",encoding="utf-8") as f:
                    flatlines.to_json(f,orient="records",lines=True,index=False)
                    i = i + 1
                    if i>=4 :
                        break

In [7]:
%ls -l input | wc -l

5


In [8]:
# design schema here
dSchema = T.StructType([\
   T.StructField("$Schema", T.StringType())\
  ,T.StructField("id", T.IntegerType())\
  ,T.StructField("type", T.StringType())\
  ,T.StructField("namespace", T.IntegerType())\
  ,T.StructField("title", T.StringType())\
  ,T.StructField("title_url", T.StringType())\
  ,T.StructField("comment", T.StringType())\
  ,T.StructField("timestamp", T.IntegerType())\
  ,T.StructField("user", T.StringType())\
  ,T.StructField("bot", T.BooleanType())\
  ,T.StructField("log_id", T.IntegerType())\
  ,T.StructField("log_type", T.StringType())\
  ,T.StructField("log_action", T.StringType())\
  ,T.StructField("log_action_comment", T.StringType())\
  ,T.StructField("server_url", T.StringType())\
  ,T.StructField("server_name", T.StringType())\
  ,T.StructField("server_script_path", T.StringType())\
  ,T.StructField("wiki", T.StringType())\
  ,T.StructField("parsedcomment", T.StringType())\
  ,T.StructField("meta_uri", T.StringType())\
  ,T.StructField("meta_request_id", T.StringType())\
  ,T.StructField("meta_id", T.StringType())\
  ,T.StructField("meta_dt", T.TimestampType())\
  ,T.StructField("meta_domain", T.StringType())\
  ,T.StructField("meta_stream", T.StringType())\
  ,T.StructField("meta_topic", T.StringType())\
  ,T.StructField("meta_partition", T.IntegerType())\
  ,T.StructField("meta_offset", T.IntegerType())\
  ,T.StructField("log_params_userid", T.IntegerType())\
                      ])

In [9]:
spark.read.load("input",format="json",schema=dSchema).select("meta_dt","type").orderBy("meta_dt", ascending=False).show(15)

+-------------------+----------+
|            meta_dt|      type|
+-------------------+----------+
|2024-09-16 06:57:53|      edit|
|2024-09-16 06:57:49|categorize|
|2024-09-16 06:57:40|categorize|
|2024-09-16 06:57:40|categorize|
+-------------------+----------+



# Folder clean up

In [ ]:
%rm -f input/*.json

In [ ]:
%rm -rf *check*

In [ ]:
%rm -rf *output*

# Structured Streaming

In [10]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
recentChange = spark.readStream.format("json").schema(dSchema).load("input")


In [ ]:
recentChange.isStreaming

True

In [11]:
rc_df = recentChange.withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))
rc_df

DataFrame[$Schema: string, id: int, type: string, namespace: int, title: string, title_url: string, comment: string, timestamp: int, user: string, bot: boolean, log_id: int, log_type: string, log_action: string, log_action_comment: string, server_url: string, server_name: string, server_script_path: string, wiki: string, parsedcomment: string, meta_uri: string, meta_request_id: string, meta_id: string, meta_dt: timestamp, meta_domain: string, meta_stream: string, meta_topic: string, meta_partition: int, meta_offset: int, log_params_userid: int]

In [ ]:
# Assuming 'recentChange' is your streaming DataFrame
myquery = recentChange\
  .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
  .withWatermark("meta_dt", "1 minute")\
  .groupBy(F.window("meta_dt","1 minute")\
         , "type")\
  .count()\
  .writeStream\
  .format("json")\
  .outputMode("append")\
  .option("path", "output")\
  .option("checkpointLocation", "checkpoint_dir")\
  .start()



In [ ]:
# Assuming 'recentChange' is your streaming DataFrame; This cell is working without aggregation
# myquery = recentChange\
#   .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
#   .writeStream\
#   .format("csv")\
#   .outputMode("append")\
#   .option("path", "output")\
#   .option("header","true")\
#   .option("checkpointLocation", "checkpoint_dir")\
#   .start()

In [ ]:
%cat output/part-00093-b19746f6-e84a-4424-b8bc-b0468383475b-c000.json

{"window":{"start":"2024-09-13T11:16:00.000Z","end":"2024-09-13T11:17:00.000Z"},"type":"edit","count":3}


In [ ]:
%cat output/part-00196-0b21f675-e29e-4c82-b31a-dedbd3990186-c000.json

{"window":{"start":"2024-09-13T11:16:00.000Z","end":"2024-09-13T11:17:00.000Z"},"type":"log","count":1}


In [ ]:
%cat output/part-00047-9d86922b-7bea-4cdc-8440-aa6aa0131d09-c000.json

{"window":{"start":"2024-09-13T11:21:00.000Z","end":"2024-09-13T11:22:00.000Z"},"type":"edit","count":4}


In [ ]:
%ls -lrt output

In [ ]:
spark.read.load("output",format="csv",schema = T.StructType(["window"])).show(15)

In [ ]:
myquery.stop()

In [ ]:
inmemoryQuery = recentChange\
  .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
  .groupBy(F.window("meta_dt","1 minute"), "type")\
  .count()\
  .writeStream\
  .queryName("myquery")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [ ]:
spark.sql("select * from myquery order by type").show(truncate=False)

In [ ]:
inmemoryQuery.stop()

In [ ]:
%ls -lr input

#Testing

In [ ]:
rate_df = spark.readStream.format("rate").option("rowsPerSecond", 100).load()

In [ ]:
rate_df.printSchema()

In [ ]:
rate_df.writeStream.format("memory").queryName("query_name").outputMode("append").start()

In [ ]:
spark.sql("select * from query_name").show(truncate=False)

In [ ]:
json_rate_df=rate_df.withWatermark("timestamp", "5 minutes")\
.groupBy(F.window("timestamp","1 minutes"))\
.count()\
.writeStream.format("json").option("path", "json_output").option("checkpointLocation", "checkpoint_dir_json")\
.trigger(processingTime="3 minutes")\
.start()

In [ ]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-11 15:05:24'

In [ ]:
%ls --full-time -lrt  json_output

In [ ]:
json_rate_df.stop()

In [ ]:
%cat json_output/part-00002-70c4bd8c-ed49-402f-a95a-6c4545de077e-c000.json

In [ ]:
%cat json_output/part-00091-42089495-424d-444d-9ede-f083acfd61dd-c000.json

In [ ]:
%cat json_output/part-00075-f8c9962f-45e4-47ac-9a38-767593f31c98-c000.json

In [14]:
table_query = rc_df.writeStream\
  .format("parquet")\
  .option("checkpointLocation", "checkpoint_dir_parquet")\
  .toTable("rc_table")

In [21]:
spark.sql("select * from rc_table").show()

+-------+----------+----------+---------+--------------------+--------------------+--------------------+----------+--------------------+-----+------+--------+----------+------------------+--------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------+-----------+-----------------+
|$Schema|        id|      type|namespace|               title|           title_url|             comment| timestamp|                user|  bot|log_id|log_type|log_action|log_action_comment|          server_url|         server_name|server_script_path|        wiki|       parsedcomment|            meta_uri|     meta_request_id|             meta_id|            meta_dt|         meta_domain|         meta_stream|          meta_topic|meta_partition|meta_offset|log_params_userid|
+-------+----------+----------+---------+---------

In [19]:
table_query.stop()

In [22]:
table_agg_query = rc_df\
  .withWatermark("meta_dt", "1 minute")\
  .groupBy(F.window("meta_dt","1 minute")\
         , "type")\
  .count()\
  .writeStream\
  .format("parquet")\
  .option("checkpointLocation", "checkpoint_dir_table_agg)")\
  .toTable("rc_table_agg")

In [28]:
spark.sql("select * from rc_table_agg").show(truncate=False)

+------------------------------------------+----------+-----+
|window                                    |type      |count|
+------------------------------------------+----------+-----+
|{2024-09-16 06:57:00, 2024-09-16 06:58:00}|categorize|3    |
|{2024-09-16 07:05:00, 2024-09-16 07:06:00}|categorize|2    |
|{2024-09-16 06:57:00, 2024-09-16 06:58:00}|edit      |1    |
|{2024-09-16 07:05:00, 2024-09-16 07:06:00}|edit      |2    |
+------------------------------------------+----------+-----+

